In [16]:
import torch
import torch.nn as nn


In [17]:
##reusing the multiheadattention class
class MultiHeadAttention(nn.Module):
  def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
    super().__init__()
    assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads
    self.w_query = nn.Linear(d_in,d_out,qkv_bias)
    self.w_key = nn.Linear(d_in,d_out,qkv_bias)
    self.w_value = nn.Linear(d_in,d_out,qkv_bias)
    self.out_proj = nn.Linear(d_out,d_out) # Corrected d_in to d_out
    self.dropout = nn.Dropout(dropout)
    self.register_buffer(
        "mask",
        torch.triu(torch.ones(context_length,context_length),diagonal=1)
    )

  def forward(self,x):
    b,num_tokens,d_in = x.shape
    keys = self.w_key(x)
    values = self.w_value(x)
    queries = self.w_query(x)
    ##splitting matrix by adding a num_heads and head_dim dimensions
    keys= keys.view(b,num_tokens,self.num_heads,self.head_dim)
    values = values.view(b,num_tokens,self.num_heads,self.head_dim)
    queries = queries.view(b,num_tokens,self.num_heads,self.head_dim)

    #tranpsoing from shape (b,num_tokens,num_heads,head_dim) to (b,num_heads,num_tokens,head_dim)
    keys = keys.transpose(1,2)
    values = values.transpose(1,2)
    queries = queries.transpose(1,2)

    attn_scores = queries @ keys.transpose(2,3) # Fixed typo tranpose -> transpose
    mask_bool = self.mask.bool()[:num_tokens, :num_tokens].unsqueeze(0).unsqueeze(0) # Added unsqueeze for broadcasting

    attn_scores.masked_fill_(mask_bool,-torch.inf)

    attn_weights = torch.softmax(
        attn_scores /keys.shape[-1]**0.5,dim=-1
    )
    attn_weights = self.dropout(attn_weights)

    context_vec_per_head = (attn_weights @ values).transpose(1,2) # Fixed typo tranpose -> transpose and renamed variable

    context_vec = context_vec_per_head.contiguous().view( # Used corrected variable name
        b,num_tokens,self.d_out
    )
    context_vec = self.out_proj(context_vec)
    return context_vec


In [18]:
## layernorm class
class LayerNorm(nn.Module):
  def __init__(self,emb_dim):
    super().__init__()
    self.eps = 1e-5
    self.scale = nn.Parameter(torch.ones(emb_dim))
    self.shift = nn.Parameter(torch.zeros(emb_dim))

  def forward(self,x):
    mean = x.mean(dim=-1,keepdim=True)
    var = x.var(dim=-1,keepdim=True)
    norm_x = (x - mean) / torch.sqrt(var + self.eps)
    return self.scale * norm_x + self.shift


In [19]:
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return 0.5 * x * (1 + torch.tanh(
        torch.sqrt(torch.tensor(2.0/torch.pi)) * (x + 0.044715 * torch.pow(x,3))
    ))


In [20]:
class FeedForward(nn.Module):
  def __init__(self,cfg):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(cfg["emb_dim"],4*cfg["emb_dim"]),
        GELU(),
        nn.Linear(4*cfg["emb_dim"], cfg["emb_dim"])
    )

  def forward(self,x):
    return self.layers(x)


In [21]:
class TransformerBlock(nn.Module):
  def __init__(self,cfg):
    super().__init__()
    self.att = MultiHeadAttention(
        d_in=cfg["emb_dim"],
        d_out = cfg["emb_dim"],
        context_length = cfg["context_size"],
        num_heads = cfg["n_heads"],
        dropout = cfg["drop_rate"],
        qkv_bias = cfg["qkv_bias"]

    )
    self.ffn = FeedForward(cfg)
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

  def forward(self,x):
    shortcut = x
    x = self.norm1(x)
    x = self.att(x)
    x = self.drop_shortcut(x) # Fixed typo: dropout_shortcut -> drop_shortcut
    x = x + shortcut

    shortcut = x
    x = self.norm2(x)
    x = self.ffn(x) # Fixed typo: ff -> ffn
    x = self.drop_shortcut(x)
    x = x + shortcut
    return x

In [22]:
class GPTModel(nn.Module):
  def __init__(self,cfg):
    super().__init__()
    self.tok_emb = nn.Embedding(cfg["vocab_size"],cfg["emb_dim"])
    self.pos_emb = nn.Embedding(cfg["context_size"],cfg["emb_dim"]) # Changed context_length to context_size
    self.drop_emb = nn.Dropout(cfg["drop_rate"])

    self.trf_blocks = nn.Sequential(
        *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]

    )
    self.final_norm = LayerNorm(cfg["emb_dim"])
    self.out_head = nn.Linear(
        cfg["emb_dim"], cfg["vocab_size"],bias=False
    )

  def forward(self,in_idx):
    batch_size,seq_len = in_idx.shape
    tok_embeds= self.tok_emb(in_idx)
    pos_embeds = self.pos_emb(
        torch.arange(seq_len,device=in_idx.device)
    )
    x = tok_embeds + pos_embeds
    x = self.drop_emb(x) # Corrected dropout application
    x = self.trf_blocks(x)
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits

In [24]:
#from IPython.utils.process import get_output_error_code
GPT_CONFIG_124M = {
    "vocab_size":50257,
    "context_size":256,
    "emb_dim":768,
    "n_heads":12,
    "n_layers":12,
    "drop_rate":0.1,
    "qkv_bias":False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (w_query): Linear(in_features=768, out_features=768, bias=False)
        (w_key): Linear(in_features=768, out_features=768, bias=False)
        (w_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffn): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (w_query): Linear(in_feature

* Here we have made some few adjustment to the `GPT_CONFIG_124M` dictionary by reducing the context size to 256 tokens.
* This modification reduces the computatioal demands of training the model, making it possible to carry out the training on a standard laptop cmputer

In [29]:
def generate_text(model,idx,max_new_tokens,context_size):
  for _ in range(max_new_tokens):
    # Crop idx to the last context_size tokens if it's longer
    idx_cond = idx[:,  -context_size:]
    with torch.no_grad():
      logits = model(idx_cond)

    logits = logits[:,-1,:]
    probs = torch.softmax(logits,dim=-1)
    idx_next = torch.argmax(probs,dim=-1,keepdim=True)
    idx = torch.cat((idx,idx_next),dim=1)
  return idx

In [31]:
##implementing text generation process
import tiktoken

def text_to_token_ids(text,tokenizer):
  encoded = tokenizer.encode(text,allowed_special={'<|endoftext|>'})
  encoded_tensor = torch.tensor(encoded).unsqueeze(0)# unsqueeze adds the batch dimension
  return encoded_tensor

def token_ids_to_text(token_ids,tokenizer):
  flat = token_ids.squeeze(0) #remove batch dimension
  return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text(
    model=model,
    idx=text_to_token_ids(start_context,tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_size"]
)
print("Output text:\n",token_ids_to_text(token_ids,tokenizer))


Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


* As we can see the model isn't yet producing coherent text because it hasn't undergone training.
* To define what makes text 'coherent' or 'high quality' we have to implement a numerical method to evaluate the generated content.

## 1.2  Calculating the text generation loss.

* Consider these two input examples, which have already been mapped to token IDs.

In [32]:
inputs = torch.tensor(
    [
        [16833,3626,6100], #every effort moves
        [40,1107,588] # I really like
    ]
)

* Matching these inpts, the targets contain token IDs we want the model to produce.

In [40]:
targets = torch.tensor([
    [3626,6100,345],#every effort moves you
    [1107,588,11311]#really like chocolate
])

* Note that these targets are inputs but shifted one position forward. This shifting strategy is crucial for teaching the model to predict the next token in a sequence.
* Next we feedthe inputs into the model to calculate logits vectors for the two input examples. The we apply the `softmax` function to transform these logits into probability scores.

In [41]:
with torch.no_grad():
  logits = model(inputs)
probs = torch.softmax(logits,dim=-1)
print(probs.shape)

torch.Size([2, 3, 50257])


In [42]:
##applying the argmax function to the probability scores to obtain the correspoding token IDs
token_ids = torch.argmax(probs,dim=-1,keepdim=True)
print("Token IDs:\n",token_ids)

Token IDs:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


* Given that we have two input batches, each containing three tokens, applying `argmax` function to the probability scores yields two set of outputs, each with three predicted token Ids

In [44]:
##converting the token IDs back into text
print(f"Target batch1:{token_ids_to_text(targets[0],tokenizer)}")
print(f"Outputs batch1:"f"{token_ids_to_text(token_ids[0].flatten(),tokenizer)}")

Target batch1: effort moves you
Outputs batch1: Armed heNetflix


* As we can see the model produces random text that is different from the target text because it has not been trained yet.
* Model training aims to increase the softmax probability in the index postions corresponding to the correct target token Ids.
* This softmax probability is also used in evaluation metric we will implement next to numerically access the model's generated outputs: higher probability in the correct positions, the better.

In [45]:
text_idx = 0
target_probs1 = probs[text_idx,[0,1,2],targets[text_idx]]
print("Text 1:",target_probs1)

text_idx = 1
target_probs2 = probs[text_idx,[0,1,2],targets[text_idx]]
print("Text 2: ",target_probs2)

Text 1: tensor([7.4514e-05, 3.1054e-05, 1.1567e-05])
Text 2:  tensor([1.0343e-05, 5.6737e-05, 4.7620e-06])


* The goal of training an LLM is to maximize relative likelihood of the current token, which involves increasing its probability relative to other tokens.
* This way, we ensure the LLM consistently picks the target token i.e the next word in the sentence as the next tokem it generates.

In [46]:
##calculating probability scores of the two example batches
log_probs =torch.log(torch.cat((target_probs1,target_probs2)))
print(log_probs)

tensor([ -9.5045, -10.3798, -11.3674, -11.4792,  -9.7771, -12.2549])


* Working with logarithms of the probabilities scores is more managable in mathematical optimization than handling the scores directly.

In [48]:
##combining the log probabilities into  single score by computing the averagr
avg_log_probs = torch.mean(log_probs)
print(avg_log_probs)

tensor(-10.7938)


* The goal is to get the average log probability as close to 0 as possilbe by updating the model's weights as part of the training process.
* However, in deep learning, the common practice isn't to push the average log probability up to 0 but rather to bring the negative average log probability to 0.
* The negative average log probability is simply the average log probability multiplied by -1 .

In [50]:
neg_avg_log_probs = avg_log_probs *-1
print(neg_avg_log_probs)

tensor(10.7938)


In [51]:
#recalling shape of the logits and target tensors
print("Logits shape: ",logits.shape) #dim=batch_size,num_tokens,vocab_size
print("Target shape: ",targets.shape)#batch_size and num_tokens

Logits shape:  torch.Size([2, 3, 50257])
Target shape:  torch.Size([2, 3])


In [52]:
##flattening tensors by combining them over the batch dimension
logits_flat = logits.flatten(0,1)
targets_flat = targets.flatten()
print("Flattened logits: ",logits_flat.shape)
print("Flattened targets: ",targets_flat.shape)

Flattened logits:  torch.Size([6, 50257])
Flattened targets:  torch.Size([6])


* Previously, we applied the softmax function, selected the probability scores corresponding to the target IDs, and computed the negative average log probabilites.
* Pytorch's `cross_entropy` function will take care of the these steps.

In [54]:
loss= torch.nn.functional.cross_entropy(logits_flat,targets_flat)
print(loss)

tensor(10.7938)


* The above resulting loss is the same that we obtained previously when applying the individual steps manually.

## 1.3 Calculating the trainig and validation set losses.

* We must first prepare the training and validation datasets that we will use to train the LLM.
* To compute loss, we use a very samll text dataset, the "Notes from the underground"

In [55]:
import urllib.request
url = "https://www.gutenberg.org/cache/epub/600/pg600.txt"
input_file ="pg600.txt"
urllib.request.urlretrieve(url, input_file)

('pg600.txt', <http.client.HTTPMessage at 0x79b3221455b0>)

In [57]:
with open(input_file,'r',encoding='utf-8') as f:
  text_data = f.read()

In [59]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters: ",total_characters)
print("Tokens:",total_tokens)

Characters:  259118
Tokens: 67308


In [61]:
##diving data into training and validation set
train_ratio = 0.90
split_idx = int(train_ratio*len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

* Using the `train_data` and `val_data` subsets, we can now create the respective dataloader reusing the `create_dataloader_v1` function

In [64]:
## creating a dataset for batched inputs and targets
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)##tokenizes text

    for i in range(0,len(token_ids) - max_length,stride):#using sliding window to chuk the book into overlapping sequences of max_length
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx], self.target_ids[idx]

In [65]:
## creating a dataloader to generate batches with input-with pairs
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
  dataloader = DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )
  return dataloader

In [66]:
torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_size"],
    stride=GPT_CONFIG_124M["context_size"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)
val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_size"],
    stride=GPT_CONFIG_124M["context_size"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [67]:
print("Train loader")
for x,y in train_loader:
  print(x.shape,y.shape)
print("\nValidation loader:")
for x,y in val_loader:
  print(x.shape,y.shape)

Train loader
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) 

In [68]:
##implementing a utility function to calculate the cross entropy loss
## of a given batch returned via the training and validation loader
def calc_loss_batch(input_batch,target_batch,model,device):
  input_batch = input_batch.to(device)
  target_batch = target_batch.to(device)
  logits = model(input_batch)
  loss = torch.nn.functional.cross_entropy(
      logits.flatten(0,1),target_batch.flatten()
  )
  return loss

In [71]:
##implementing a function to compute the training and validation loss
def calc_loss_loader(data_loader,model,device,num_batches=None):
  total_loss = 0
  if len(data_loader) == 0:
    return float("nan")

  # Determine the actual number of batches to process
  if num_batches is None:
    actual_num_batches = len(data_loader)
  else:
    actual_num_batches = min(num_batches, len(data_loader))

  for i, (input_batch,target_batch) in enumerate(data_loader):
    if i < actual_num_batches:
      loss = calc_loss_batch(
          input_batch,target_batch,model,device
      )
      total_loss += loss.item()
    else:
      break
  # Avoid division by zero if actual_num_batches is 0
  if actual_num_batches == 0:
    return float("nan")
  return total_loss / actual_num_batches ##averaging loss over all batches


In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with torch.no_grad():
  train_loss = calc_loss_loader(train_loader,model,device)
  val_loss = calc_loss_loader(val_loader,model,device)
print("Training loss:", train_loss)
print("Validation loss:",val_loss)

Training loss: 10.979752308180352
Validation loss: 10.972686608632406


* The loss values are relatively high because the model has not yet been trainied.

## 1.4 Training the LLM to generate human like text


In [75]:
## main function for pretraining LLMs
def train_model(model,train_loader,val_loader,optimizer,device,num_epochs,eval_freq,eval_iter,start_context,tokenizer):
  train_losses,val_losses,track_tokens_seen = [], [], []
  tokens_seen,global_step = 0, -1

  for epoch in range(num_epochs):
    model.train()
    for input_batch,target_batch in train_loader:
      optimizer.zero_grad() #rest loss gradients from previous batch iterations
      loss = calc_loss_batch(
          input_batch,target_batch,model,device
      )
      loss.backward()
      optimizer.step()
      tokens_seen += input_batch.numel()
      global_step += 1

      if global_step % eval_freq == 0:
        train_loss,val_loss = evaluate_model(
            model,train_loader,val_loader,device,eval_iter
        )
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        track_tokens_seen.append(tokens_seen)
        print(
            f"Ep {epoch+1} (Step {global_step:06d}):"
            f"Train Loss {train_loss:.3f},"
            f"Val loss {val_loss:.2f}"
        )
    generate_sample(
       model,tokenizer,device,start_context
   )
    return train_losses,val_losses,track_tokens_seen

In [76]:
def evaluate_model(model,train_loader,val_loader,eval_iter):
  model.eval()
  with torch.no_grad():
    train_loss = calc_loss_header(
        train_loader,model,device,num_batches=eval_iter
    )
    val_loss = calc_loss_loader(
        val_loader,model,device,num_batces=eval_iter
    )
  model.train()
  return train_loss,val_loss

In [ ]:
def generate(model,tokenizer,device,start_context):
  model.eval()
  context_size = model.pos_emb.weight.shape[0]
  encoded = text_to_token_ids(start_context,tokenizer).to(device)
  with torch.no_grad():
    token_ids = generate_text(
        model=model,
        idx=encoded,
        max_new_tokens=50,
        context_size=context_size
    )
  decoded_text =token_ids_to_text(tokens_ids,tokenizer)
  print(decoded_text.replace("\n"," "))
  model.train()